In [2]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier

In [3]:
train_df = pd.read_csv('train_df.csv', index_col=0)
train_df.drop(['positive', 'pos_corpus', 'neg_corpus', 'pos_1', 'pos_2', 'pos_3', 
               'neg_1', 'neg_2', 'neg_3', 'prod_corpus'], axis=1, inplace=True)
#train_df.drop(['cust_total_votes_mean', 'cust_total_votes_std', 'cust_helpful_votes_mean', 'cust_helpful_votes_std', 
#               'prod_total_votes_mean', 'prod_total_votes_std', 'prod_helpful_votes_mean', 'prod_helpful_votes_std'], 
#              axis=1, inplace=True)
train_df.head()

,customer_id,product_parent,star_rating,cust_review_count,cust_star_rating_mean,cust_star_rating_std,cust_total_votes_mean,cust_total_votes_std,cust_helpful_votes_mean,cust_helpful_votes_std,prod_review_count,prod_star_rating_mean,prod_star_rating_std,prod_total_votes_mean,prod_total_votes_std,prod_helpful_votes_mean,prod_helpful_votes_std,pos_sim,neg_sim
0,1617361,849246716,5,5,5.000000,0.000000,0.200000,0.447214,0.000000,0.000000,297,4.872054,0.363625,0.424242,3.224979,0.37037,3.042081,0.033451,0.000000
1,53052607,849246716,3,5,3.600000,0.547723,0.400000,0.894427,0.400000,0.894427,297,4.872054,0.363625,0.424242,3.224979,0.37037,3.042081,0.012478,0.009009
2,15679577,849246716,5,8,3.375000,1.505941,7.625000,19.175412,6.500000,17.188036,297,4.872054,0.363625,0.424242,3.224979,0.37037,3.042081,0.007220,0.033333
3,16367779,849246716,5,9,4.444444,0.527046,0.888889,1.364225,0.777778,1.092906,297,4.872054,0.363625,0.424242,3.224979,0.37037,3.042081,0.095238,0.000000
4,25485198,849246716,5,17,5.000000,0.000000,0.235294,0.437237,0.176471,0.392953,297,4.872054,0.363625,0.424242,3.224979,0.37037,3.042081,0.010870,0.000000


In [4]:
train_df.groupby('star_rating').size()

star_rating
1     4928
2     3324
3     5475
4    11451
5    43771
dtype: int64

In [5]:
temp = train_df[train_df.star_rating == 1][:3324]
temp = pd.concat([temp, train_df[train_df.star_rating == 2]])
temp = pd.concat([temp, train_df[train_df.star_rating == 3]])
temp = pd.concat([temp, train_df[train_df.star_rating == 4][:len(train_df[train_df.star_rating == 4]) // 2]])
temp = pd.concat([temp, train_df[train_df.star_rating == 5][:len(train_df[train_df.star_rating == 5]) // 8]])
train_df = temp

In [6]:
test_df = pd.read_csv('test_df.csv', index_col=0)
test_df.drop(['positive', 'pos_corpus', 'neg_corpus', 'pos_1', 'pos_2', 'pos_3', 
               'neg_1', 'neg_2', 'neg_3', 'prod_corpus'], axis=1, inplace=True)
#test_df.drop(['cust_total_votes_mean', 'cust_total_votes_std', 'cust_helpful_votes_mean', 'cust_helpful_votes_std', 
#               'prod_total_votes_mean', 'prod_total_votes_std', 'prod_helpful_votes_mean', 'prod_helpful_votes_std'], 
#              axis=1, inplace=True)
test_df.head()

,customer_id,product_parent,star_rating,cust_review_count,cust_star_rating_mean,cust_star_rating_std,cust_total_votes_mean,cust_total_votes_std,cust_helpful_votes_mean,cust_helpful_votes_std,prod_review_count,prod_star_rating_mean,prod_star_rating_std,prod_total_votes_mean,prod_total_votes_std,prod_helpful_votes_mean,prod_helpful_votes_std,pos_sim,neg_sim
0,36330222,986428010,5,24,4.291667,1.267629,0.541667,1.250362,0.333333,1.049500,18,4.333333,1.188177,0.833333,2.572479,0.722222,2.346601,0.070423,0.012295
1,24360083,986428010,4,23,4.347826,1.191206,1.434783,2.191431,1.000000,1.537412,18,4.333333,1.188177,0.833333,2.572479,0.722222,2.346601,0.095361,0.049808
2,28891040,437083384,4,10,4.500000,0.707107,0.000000,0.000000,0.000000,0.000000,407,4.503686,0.887439,12.356265,112.878540,10.651106,100.388954,0.006944,0.005208
3,52449052,437083384,3,5,3.400000,1.516575,11.800000,20.789420,10.400000,18.187908,407,4.503686,0.887439,12.356265,112.878540,10.651106,100.388954,0.010390,0.059514
4,27192976,437083384,4,6,4.666667,0.816497,0.500000,1.224745,0.000000,0.000000,407,4.503686,0.887439,12.356265,112.878540,10.651106,100.388954,0.012942,0.003460


In [9]:
len(train_df.query('star_rating == 5')) / len(train_df)

0.23461554955186759

In [10]:
from sklearn import preprocessing
def normalize(df):
    min_max_scaler = preprocessing.MinMaxScaler()
    newdf = pd.DataFrame(min_max_scaler.fit_transform(df), columns=df.columns)

    return df

In [11]:
train_ids = train_df[['customer_id', 'product_parent', 'star_rating']]

In [12]:
from sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler()
train_df_norm = pd.DataFrame(min_max_scaler.fit_transform(train_df), columns=train_df.columns)
train_df_norm[['customer_id', 'product_parent', 'star_rating']] = train_ids
train_df_norm.head()

,customer_id,product_parent,star_rating,cust_review_count,cust_star_rating_mean,cust_star_rating_std,cust_total_votes_mean,cust_total_votes_std,cust_helpful_votes_mean,cust_helpful_votes_std,prod_review_count,prod_star_rating_mean,prod_star_rating_std,prod_total_votes_mean,prod_total_votes_std,prod_helpful_votes_mean,prod_helpful_votes_std,pos_sim,neg_sim
0,1617361.0,849246716.0,5.0,0.002481,0.400000,0.948683,0.001776,0.000871,0.001525,0.000939,0.049681,0.837340,0.384026,0.392463,0.427464,0.501969,0.419311,0.020482,0.052490
1,53052607.0,849246716.0,3.0,0.006203,0.562500,0.601338,0.013131,0.003615,0.010864,0.003203,0.000799,0.541667,0.648717,0.026097,0.005293,0.036651,0.005433,0.166667,0.207689
2,15679577.0,849246716.0,5.0,0.002481,0.750000,0.612372,0.000888,0.000359,0.000915,0.000376,0.246486,0.863666,0.355818,0.009662,0.011216,0.010488,0.010200,0.074841,0.034172
3,16367779.0,849246716.0,5.0,0.013648,0.660714,0.825212,0.002748,0.001543,0.002396,0.001346,0.246486,0.863666,0.355818,0.009662,0.011216,0.010488,0.010200,0.086514,0.100559
4,25485198.0,849246716.0,5.0,0.002481,0.950000,0.193649,0.002959,0.001582,0.002440,0.001281,0.246486,0.863666,0.355818,0.009662,0.011216,0.010488,0.010200,0.031915,0.000000


In [13]:
test_ids = test_df[['customer_id', 'product_parent', 'star_rating']] 

In [14]:
test_df_norm = pd.DataFrame(min_max_scaler.transform(test_df), columns=test_df.columns)
test_df_norm[['customer_id', 'product_parent', 'star_rating']]  = test_ids
test_df_norm.head()

,customer_id,product_parent,star_rating,cust_review_count,cust_star_rating_mean,cust_star_rating_std,cust_total_votes_mean,cust_total_votes_std,cust_helpful_votes_mean,cust_helpful_votes_std,prod_review_count,prod_star_rating_mean,prod_star_rating_std,prod_total_votes_mean,prod_total_votes_std,prod_helpful_votes_mean,prod_helpful_votes_std,pos_sim,neg_sim
0,36330222,986428010,5,0.026055,0.822917,0.548900,0.000801,0.000502,0.000508,0.000441,0.002716,0.833333,0.420084,0.011862,0.003781,0.014438,0.003540,0.211268,0.042946
1,24360083,986428010,4,0.024814,0.836957,0.515807,0.002123,0.000880,0.001525,0.000646,0.002716,0.833333,0.420084,0.011862,0.003781,0.014438,0.003540,0.286082,0.173979
2,28891040,437083384,4,0.008685,0.875000,0.306186,0.000000,0.000000,0.000000,0.000000,0.064856,0.875921,0.313757,0.175890,0.165907,0.212933,0.151462,0.020833,0.018192
3,52449052,437083384,3,0.002481,0.600000,0.656696,0.017459,0.008352,0.015857,0.007641,0.064856,0.875921,0.313757,0.175890,0.165907,0.212933,0.151462,0.031169,0.207879
4,27192976,437083384,4,0.003722,0.916667,0.353553,0.000740,0.000492,0.000000,0.000000,0.064856,0.875921,0.313757,0.175890,0.165907,0.212933,0.151462,0.038827,0.012086


In [15]:
train_x, train_y = train_df.drop('star_rating', axis=1), train_df['star_rating']
test_x, test_y = test_df.drop('star_rating', axis=1), test_df['star_rating']
train_x_norm = train_df_norm.drop('star_rating', axis=1)
test_x_norm = test_df.drop('star_rating', axis=1)

In [18]:
from sklearn.ensemble import GradientBoostingClassifier
gb = GradientBoostingClassifier()
gb.fit(train_x_norm.drop(['customer_id', 'product_parent'], axis=1), train_y)

GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=None, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [19]:
gb.score(test_x_norm.drop(['customer_id', 'product_parent'], axis=1), test_y)

0.1775027144408252

In [13]:
rf = RandomForestClassifier(n_estimators=1000)
rf.fit(train_x, train_y)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=1000,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [14]:
rf.score(test_x, test_y)

0.2035613463626493

In [15]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=3, weights='distance')
knn.fit(train_x_norm.drop(['customer_id', 'product_parent'], axis=1), train_y)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=3, p=2,
                     weights='distance')

In [16]:
knn.score(test_x_norm.drop(['customer_id', 'product_parent'], axis=1), test_y)

0.4824755700325733

In [17]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(train_x_norm.drop(['customer_id', 'product_parent'], axis=1), train_y)

GaussianNB(priors=None, var_smoothing=1e-09)

In [18]:
gnb.score(test_x_norm.drop(['customer_id', 'product_parent'], axis=1), test_y)

0.31465798045602605

In [20]:
correct = 0
pos_count = 0
star_cor = 0
preds = gb.predict(test_x_norm.drop(['customer_id', 'product_parent'], axis=1))
for i, row in test_df.iterrows():
    if (preds[i] > 3):
        pos_count += 1
    if (preds[i] == row.star_rating):
        star_cor += 1
    if (preds[i] > 3 and row.star_rating > 3):
        correct += 1
    elif (preds[i] <= 3 and row.star_rating <= 3):
        correct += 1

print('Correct:', correct, correct / len(test_df))
print('Pos:', pos_count, pos_count / len(test_df))
print('Star Correct:', star_cor, star_cor / len(test_df))

Correct: 7735 0.33593919652551574
Pos: 3492 0.15166123778501628
Star Correct: 4087 0.1775027144408252


In [14]:
from keras import models
from keras import layers
from keras import optimizers
from keras.utils import to_categorical
import numpy as np
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [15]:
nn_train_x = train_x_norm.drop(['customer_id', 'product_parent'], axis=1)
nn_test_x = test_x_norm.drop(['customer_id', 'product_parent'], axis=1)

In [16]:
np.shape(nn_train_x)

(23319, 16)

In [17]:
train_y_cat = to_categorical(train_y)
test_y_cat = to_categorical(test_y)

In [18]:
network = models.Sequential()
network.add(layers.Dense(1024, activation='relu', input_shape=(16,)))
#network.add(layers.Dropout(0.5))
network.add(layers.Dense(1024, activation='relu', input_shape=(1024,)))
network.add(layers.Dense(1024, activation='relu', input_shape=(1024,)))
network.add(layers.Dense(512, activation='relu', input_shape=(1024,)))
#network.add(layers.Dropout(0.5))
network.add(layers.Dense(512, activation='relu', input_shape=(512,)))
network.add(layers.Dense(512, activation='relu', input_shape=(512,)))
network.add(layers.Dense(256, activation='relu', input_shape=(512,)))
#network.add(layers.Dropout(0.5))
network.add(layers.Dense(256, activation='relu', input_shape=(256,)))
network.add(layers.Dense(256, activation='relu', input_shape=(256,)))
network.add(layers.Dense(128, activation='relu', input_shape=(256,)))
#network.add(layers.Dropout(0.5))
network.add(layers.Dense(128, activation='relu', input_shape=(128,)))
network.add(layers.Dense(128, activation='relu', input_shape=(128,)))
network.add(layers.Dense(6, activation='softmax', input_shape=(128,)))
network.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 1024)              17408     
_________________________________________________________________
dense_2 (Dense)              (None, 1024)              1049600   
_________________________________________________________________
dense_3 (Dense)              (None, 1024)              1049600   
_________________________________________________________________
dense_4 (Dense)              (None, 512)               524800    
_________________________________________________________________
dense_5 (Dense)              (None, 512)               262656    
_________________________________________________________________
dense_6 (Dense)              (None, 512)               262656    
_________________________________________________________________
dense_7 (Dense)              (None, 256)              

In [ ]:
opt = optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, amsgrad=False)
network.compile(optimizer=opt,
                loss='categorical_crossentropy',
                metrics=['accuracy'])

epochs = 150
history = network.fit(nn_train_x, 
                      train_y_cat, 
                      epochs=epochs, 
                      batch_size=128, 
                      validation_data=(nn_test_x, test_y_cat))

Train on 23319 samples, validate on 23025 samples
Epoch 1/150
23319/23319 [==============================] - 10s 436us/step - loss: 1.4749 - accuracy: 0.3422 - val_loss: 171.1147 - val_accuracy: 0.5690
Epoch 2/150
23319/23319 [==============================] - 2s 70us/step - loss: 1.3980 - accuracy: 0.3904 - val_loss: 118.5110 - val_accuracy: 0.5548
Epoch 3/150
23319/23319 [==============================] - 1s 64us/step - loss: 1.3753 - accuracy: 0.4095 - val_loss: 79.7120 - val_accuracy: 0.5431
Epoch 4/150
23319/23319 [==============================] - 2s 68us/step - loss: 1.3646 - accuracy: 0.4175 - val_loss: 102.2787 - val_accuracy: 0.5623
Epoch 5/150
23319/23319 [==============================] - 2s 66us/step - loss: 1.3702 - accuracy: 0.4170 - val_loss: 45.3590 - val_accuracy: 0.4202
Epoch 6/150
23319/23319 [==============================] - 2s 66us/step - loss: 1.3523 - accuracy: 0.4270 - val_loss: 149.0704 - val_accuracy: 0.3669
Epoch 7/150
23319/23319 [=========================

23319/23319 [==============================] - 2s 79us/step - loss: 1.2584 - accuracy: 0.4704 - val_loss: 1839.7621 - val_accuracy: 0.3196
Epoch 56/150
23319/23319 [==============================] - 2s 70us/step - loss: 1.2555 - accuracy: 0.4703 - val_loss: 965.9140 - val_accuracy: 0.3250
Epoch 57/150
 7424/23319 [========>.....................] - ETA: 0s - loss: 1.2468 - accuracy: 0.4759

In [ ]:
history_dict = history.history
loss_values = history_dict['loss']
test_loss_values = history_dict['val_loss']
epochs_range = range(1, epochs + 1)
plt.plot(epochs_range, loss_values, 'bo', label='Training loss')
plt.plot(epochs_range, test_loss_values, 'ro', label='Test loss')
plt.title('Training and test loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
acc_values = history_dict['accuracy']
test_acc_values = history_dict['val_accuracy']
plt.plot(epochs_range, acc_values, 'bo', label='Training accuracy')
plt.plot(epochs_range, test_acc_values, 'ro', label='Test accuracy')
plt.title('Training and test accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
correct = 0
pos_count = 0
star_cor = 0
preds = network.predict(nn_test_x)
for i, row in test_df.iterrows():
    if (np.argmax(preds[i]) > 3):
        pos_count += 1
    if (np.argmax(preds[i]) == np.argmax(test_y_cat[i])):
        star_cor += 1
    if (np.argmax(preds[i]) > 3 and np.argmax(test_y_cat[i]) > 3):
        correct += 1
    elif (np.argmax(preds[i]) <= 3 and np.argmax(test_y_cat[i]) <= 3):
        correct += 1

print('Correct:', correct, correct / len(test_df))
print('Pos:', pos_count, pos_count / len(test_df))
print('Star Correct:', star_cor, star_cor / len(test_df))

In [ ]:
#network.save('basic_neural_net2.h5')